In [53]:
class FingerprintKey:
    def __init__(self, key, ti):
        self.key = key # (fi, fj, delta_t)
        self.ti = ti # Absolute start point

class SongRecord:
    def __init__(self, song, ti):
        self.song = song
        self.ti = ti # Relative (Original_song.ti - Sample_Recording.ti)
    
    def __repr__(self):
        return "(" + str(self.song) + " " + str(self.ti) + ")"

In [54]:
# key: tuple of (f_i, f_j, t_j - t_i)
# value: [("Song-A", t_i), ..., ("Song-B", t_i)]

fanout = 4

# f = 100, t = 10
# f = 150, t = 12
# f = 300, t = 15
# f = 70,  t = 23
# f = 90,  t = 27
# f = 250, t = 36
# f = 171, t = 37
# f = 52,  t = 50
# f = 314, t = 62

song_names = ["Song-A", "Song-B", "Song-C"]

f_t_values = [[(100, 10), (150, 12), (300, 15), (70, 23), (90, 27), (250, 36), (171, 37), (52, 50), (314, 62)],
              [(50, 7), (200, 8), (92, 25), (10, 30), (100, 33), (228, 48), (342, 70), (57, 82), (159, 90)],
              [(115, 14), (76, 23), (100, 30), (150, 32), (1002, 34), (921, 36), (83, 40), (37, 42), (265, 64)]]

N_songs = len(song_names)

database = {}

for s in range(len(song_names)):
    N = len(f_t_values[s])
    song_name = song_names[s]
    for i in range(N):
        for j in range(1, fanout+1):
            if i+j >= N:
                break
            f_i = f_t_values[s][i]
            f_j = f_t_values[s][i+j]

            dt = f_j[1] - f_i[1]
            key = (f_i[0], f_j[0], dt)
            fpk = FingerprintKey(key, f_i[1]) # fingerprint key
            sr = SongRecord(song_name, f_i[1]) # song record

            if fpk.key in database:
                database[fpk.key].append(sr)
            else:
                database[fpk.key] = [sr]


In [55]:
database

{(100, 150, 2): [(Song-A 10), (Song-C 30)],
 (100, 300, 5): [(Song-A 10)],
 (100, 70, 13): [(Song-A 10)],
 (100, 90, 17): [(Song-A 10)],
 (150, 300, 3): [(Song-A 12)],
 (150, 70, 11): [(Song-A 12)],
 (150, 90, 15): [(Song-A 12)],
 (150, 250, 24): [(Song-A 12)],
 (300, 70, 8): [(Song-A 15)],
 (300, 90, 12): [(Song-A 15)],
 (300, 250, 21): [(Song-A 15)],
 (300, 171, 22): [(Song-A 15)],
 (70, 90, 4): [(Song-A 23)],
 (70, 250, 13): [(Song-A 23)],
 (70, 171, 14): [(Song-A 23)],
 (70, 52, 27): [(Song-A 23)],
 (90, 250, 9): [(Song-A 27)],
 (90, 171, 10): [(Song-A 27)],
 (90, 52, 23): [(Song-A 27)],
 (90, 314, 35): [(Song-A 27)],
 (250, 171, 1): [(Song-A 36)],
 (250, 52, 14): [(Song-A 36)],
 (250, 314, 26): [(Song-A 36)],
 (171, 52, 13): [(Song-A 37)],
 (171, 314, 25): [(Song-A 37)],
 (52, 314, 12): [(Song-A 50)],
 (50, 200, 1): [(Song-B 7)],
 (50, 92, 18): [(Song-B 7)],
 (50, 10, 23): [(Song-B 7)],
 (50, 100, 26): [(Song-B 7)],
 (200, 92, 17): [(Song-B 8)],
 (200, 10, 22): [(Song-B 8)],
 (200

In [56]:
len(database[(100, 150, 2)])

2

In [57]:
my_sample = [FingerprintKey((92, 10, 5), 2), FingerprintKey((921, 265, 28), 2), FingerprintKey((1002, 921, 2), 0)]

In [70]:
from collections import Counter

def compare(sample_fingerprints, database, threshold = 0):
    '''
    Takes sample_fingerprints and matches them with database.
    Returns song that it found or None if no song was found.
    
    sample_fingerprints: A list of FingerprintKey objects for the song sample
    
    database: The database of fingerprints to compare, with the keys are FingerprintKey objects and value of SongRecord objects
    '''
    #make song_counter
    song_counter = Counter()
    for fingerprint in sample_fingerprints:
        #print("Fingerprint:", fingerprint)
        #print("Found Finger:", )
        if fingerprint.key in database: #if a match is found
            possible_songs = database[fingerprint.key]
            song_records = []
            for song_record in possible_songs: # put all the possible songs in the database
                new_song_record = (song_record.song, song_record.ti - fingerprint.ti)
                song_records.append(new_song_record)
            song_counter.update(song_records)
    print("Song Tally:", song_counter)
    result, tally = song_counter.most_common(1)[0]
    print("Highest Tally:", tally)
    print("Corresponding Song_Record object:", result)
    if tally >= threshold:
        return result[0] # return song_id
    else:
        return "Song Not Found!"

In [71]:
compare(my_sample, database)

Song Tally: Counter({('Song-C', 34): 2, ('Song-B', 23): 1})
Highest Tally: 2
Corresponding Song_Record object: ('Song-C', 34)


'Song-C'